In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

#Build a crawler to crawl Google's top news articles and pull articles containing 'Net Neutrality'
class GoogleSpider(scrapy.Spider):
    name = "GS"
    
    #response.selector.xpath('//title/text()')
    
    # Here is where we insert our API call to get the top Google news articles.
    start_urls = [
        'https://newsapi.org/v2/top-headlines?sources=google-news&apiKey={0e83adc25609438898d17835b9bb28c70e83adc25609438898d17835b9bb28c7}'
        ]

    # Identifying the information we want from the query response and extracting it using xpath.
    def parse(self, response):
        links = response.xpath('//a[contains(@href, "Net Neutrality")]')
        links.extract()
        
        for index, link in enumerate(links):
            args = (index, link.xpath('@href').extract(), link.xpath('img/@src').extract())
            print('Link number %d points to url %s and title %s' % args)
    
    
    
    
    '''
    # Identifying the information we want from the query response and extracting it using xpath.
    def parse(self, response):
        for item in response.xpath('//lh'):
            if item.xpath('//a[contains(@href, "Net Neutrality")]'):
                yield {
                    'title' : item.xpath('@title').extract_first()
                    }
        # Getting the information needed to continue to the next ten entries.
        next_page = response.xpath('continue/@lhcontinue').extract_first()
        
        # Recursively calling the spider to process the next ten entries, if they exist.
        if next_page is not None:
            next_page = '{}&lhcontinue={}'.format(self.start_urls[0],next_page)
            yield scrapy.Request(next_page, callback=self.parse)
    '''     
    
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'GoogleLinks.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'MatthewGoogleNewsCrawler (makennedy626@gmail.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    # We use CLOSESPIDER_PAGECOUNT to limit our scraper to the first 100 links.    
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

# Starting the crawler with our spider.
process.crawl(GoogleSpider)
process.start()
print('First 100 links extracted!')

First 100 links extracted!


In [2]:
import pandas as pd

# Checking whether we got data 

data=pd.read_json('GoogleLinks.json', orient='records')
print(data.shape)
print(data.tail())

ValueError: Expected object or value